# 프로젝트
## 개요
영화 시놉시스 코퍼스를 이용해, 영화 장르에 따른 상업-예술영화의 표현 편향성을 측정한다.

## 순서
- STEP 1. 형태소 분석기를 이용하여 품사가 명사인 경우 해당 단어를 추출하기
- STEP 2. 추출된 결과로 embedding model 만들기
- STEP 3. target, attribute 단어 셋 만들기
- STEP 4. WEAT score 계산과 시각화

## 달성 목표
### 1. 주어진 영화 코퍼스를 바탕으로 워드임베딩 모델 구축
- 워드임베딩의 most_similar() 메소드 결과가 의미상 바르게 나와야 한다.

### 2. 영화 구분, 장르별로 target, attribute에 대한 대표성있는 단어 셋을 생성
-	타당한 방법론을 통해 중복이 잘 제거되고 개념축을 의미적으로 잘 대표하는 단어 셋 구축

### 3. WEAT score 계산 및 시각화
- 전체 영화 장르별로 예술/일반 영화에 대한 편향성 WEAT score가 상식에 부합하는 수치로 얻어졌으며 이를 잘 시각화

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install konlpy
!pip install gensim

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 64.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 465.3/465.3 kB 47.7 MB/s eta 0:00:00


In [3]:
from konlpy.tag import Okt
from sklearn.feature_extraction.text import TfidfVectorizer
import os
import numpy as np
from functools import lru_cache
from gensim.models import Word2Vec
from concurrent.futures import ThreadPoolExecutor
import seaborn as sns
import matplotlib.pyplot as plt

class TextProcessor:
    def __init__(self, data_path):
        self.data_path = data_path
        self.okt = Okt()
        self.vectorizer = TfidfVectorizer()

    @lru_cache(maxsize=None)
    def tokenize(self, file_name):
        print(f"[{file_name} 파일 토큰화] 🔄")
        result = []
        with open(os.path.join(self.data_path, file_name), 'r', encoding='utf-8') as file:
            for line in file:
                tokenized_line = self.okt.pos(line, stem=True, norm=True)
                result.extend([word[0] for word in tokenized_line if word[1] in ["Noun"]])
        print(f"{file_name} [토큰화 완료] ✅: 총 {len(result)}개의 토큰")
        return result

    def build_word_set(self, file_name, texts, top_n=100):
        print(f"[{file_name}에 대한 대표 단어 셋 구축] 🔄")
        self.vectorizer.fit(texts)
        indices = np.argsort(self.vectorizer.idf_)[::-1]
        features = self.vectorizer.get_feature_names_out()
        top_features = [features[i] for i in indices[:top_n]]
        print(f"[{file_name}에 대한 단어 셋 구축 완료] ✅: 상위 {top_n}개 단어 선택")
        return top_features

class WEATCalculator:
    def __init__(self, model):
        self.model = model

    @staticmethod
    def cos_sim(i, j):
        return np.dot(i, j) / (np.linalg.norm(i) * np.linalg.norm(j))

    def s(self, w, A, B):
        c_a = np.mean([self.cos_sim(w, a) for a in A])
        c_b = np.mean([self.cos_sim(w, b) for b in B])
        return c_a - c_b

    def weat_score(self, X, Y, A, B):
        s_X = np.mean([self.s(x, A, B) for x in X])
        s_Y = np.mean([self.s(y, A, B) for y in Y])
        std_dev = np.std(np.concatenate([s_X, s_Y], axis=None))
        score = (s_X - s_Y) / std_dev
        return score

In [4]:
# 데이터 경로 및 장르별 파일 목록 설정
data_path = '/content/drive/MyDrive/Colab Notebooks/AIFFEL_Research_6th/going_deeper/07-09_Embedded_bias/synopsis'
genre_files = [
    'synopsis_SF.txt', 'synopsis_family.txt', 'synopsis_show.txt', 'synopsis_horror.txt', 'synopsis_etc.txt',
    'synopsis_documentary.txt', 'synopsis_drama.txt', 'synopsis_romance.txt', 'synopsis_musical.txt',
    'synopsis_mystery.txt', 'synopsis_crime.txt', 'synopsis_historical.txt', 'synopsis_western.txt',
    'synopsis_adult.txt', 'synopsis_thriller.txt', 'synopsis_animation.txt', 'synopsis_action.txt',
    'synopsis_adventure.txt', 'synopsis_war.txt', 'synopsis_comedy.txt', 'synopsis_fantasy.txt'
]

In [5]:
text_processor = TextProcessor(data_path)
genre_word_sets = {}

with ThreadPoolExecutor() as executor:
    futures = {genre: executor.submit(text_processor.tokenize, genre) for genre in genre_files}
    for genre, future in futures.items():
        tokenized_text = future.result()
        word_set = text_processor.build_word_set(genre, [' '.join(tokenized_text)], 100)
        genre_word_sets[genre] = word_set



[synopsis_SF.txt 파일 토큰화] 🔄
[synopsis_family.txt 파일 토큰화] 🔄
[synopsis_show.txt 파일 토큰화] 🔄
[synopsis_horror.txt 파일 토큰화] 🔄
[synopsis_etc.txt 파일 토큰화] 🔄
[synopsis_documentary.txt 파일 토큰화] 🔄
synopsis_family.txt [토큰화 완료] ✅: 총 8006개의 토큰
[synopsis_drama.txt 파일 토큰화] 🔄
synopsis_show.txt [토큰화 완료] ✅: 총 7984개의 토큰
[synopsis_romance.txt 파일 토큰화] 🔄
synopsis_SF.txt [토큰화 완료] ✅: 총 21712개의 토큰
[synopsis_musical.txt 파일 토큰화] 🔄
[synopsis_SF.txt에 대한 대표 단어 셋 구축] 🔄
[synopsis_SF.txt에 대한 단어 셋 구축 완료] ✅: 상위 100개 단어 선택
[synopsis_family.txt에 대한 대표 단어 셋 구축] 🔄
[synopsis_family.txt에 대한 단어 셋 구축 완료] ✅: 상위 100개 단어 선택
[synopsis_show.txt에 대한 대표 단어 셋 구축] 🔄
[synopsis_show.txt에 대한 단어 셋 구축 완료] ✅: 상위 100개 단어 선택
synopsis_musical.txt [토큰화 완료] ✅: 총 3442개의 토큰
[synopsis_mystery.txt 파일 토큰화] 🔄
synopsis_mystery.txt [토큰화 완료] ✅: 총 15726개의 토큰
[synopsis_crime.txt 파일 토큰화] 🔄
synopsis_etc.txt [토큰화 완료] ✅: 총 49025개의 토큰
[synopsis_historical.txt 파일 토큰화] 🔄
synopsis_historical.txt [토큰화 완료] ✅: 총 3620개의 토큰
[synopsis_western.txt 파일 토큰화] 🔄
synopsis_western.txt

In [6]:
tokenized_art = text_processor.tokenize('synopsis_art.txt')
tokenized_gen = text_processor.tokenize('synopsis_gen.txt')



[synopsis_art.txt 파일 토큰화] 🔄
synopsis_art.txt [토큰화 완료] ✅: 총 208471개의 토큰
[synopsis_gen.txt 파일 토큰화] 🔄
synopsis_gen.txt [토큰화 완료] ✅: 총 1008358개의 토큰


In [7]:
model = Word2Vec([tokenized_art + tokenized_gen], vector_size=100, window=5, min_count=3, sg=0)
weat_calculator = WEATCalculator(model.wv)

X = np.array([model.wv[word] for word in tokenized_art if word in model.wv])
Y = np.array([model.wv[word] for word in tokenized_gen if word in model.wv])



In [ ]:
matrix = np.zeros((len(genre_files), len(genre_files)))
for i, genre_A in enumerate(genre_word_sets.values()):
    for j, genre_B in enumerate(genre_word_sets.values()):
        if i >= j:
            continue
        A = np.array([model.wv[word] for word in genre_A if word in model.wv])
        B = np.array([model.wv[word] for word in genre_B if word in model.wv])
        score = weat_calculator.weat_score(X, Y, A, B)
        matrix[i][j] = score
        matrix[j][i] = -score



In [ ]:
genre_labels = [os.path.splitext(os.path.basename(genre))[0] for genre in genre_files]
plt.figure(figsize=(12, 10))
sns.heatmap(matrix, xticklabels=genre_labels, yticklabels=genre_labels, cmap='coolwarm', annot=True)
plt.title('Genre Bias Heatmap')
plt.xlabel('Genre A')
plt.ylabel('Genre B')
plt.show()